# 🚀 Day 1 실습 3: EXAONE 모델 QLoRA 파인튜닝

## 학습 목표
- EXAONE-3.5-2.4B-Instruct 모델을 활용한 경량 파인튜닝
- QLoRA (4-bit 양자화 + LoRA) 기법 적용
- Colab 무료 환경에서의 효율적 학습
- GPU 메모리 최적화 및 모니터링
- 체크포인트 저장 및 관리

## 시간: 14:40–16:10 (90분)

### 💡 QLoRA란?
- **Q**: Quantization (4-bit 양자화로 메모리 사용량 75% 감소)
- **LoRA**: Low-Rank Adaptation (파라미터의 0.3%만 학습)
- **결과**: 기존 대비 1/4 메모리로 비슷한 성능 달성!

## 1. 필요한 라이브러리 설치 및 Import

In [ ]:
# Day 1 실습 3: QLoRA 파인튜닝을 위한 라이브러리 설치 및 확인
print("🚀 Day 1 실습 3: EXAONE 모델 QLoRA 파인튜닝")
print("🔍 파인튜닝에 필요한 라이브러리 설치 및 확인 중...")

# 01, 02번에서 설치되지 않은 파인튜닝 전용 라이브러리들만 설치
print("📦 파인튜닝 전용 라이브러리 설치...")

# bitsandbytes 및 PEFT 관련 라이브러리들 (레거시 검증된 방법 사용)
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git

# 허깅페이스 Hub 라이브러리 추가 (모델 업로드용)
!pip install -q -U huggingface_hub

# 기타 필요한 라이브러리들 (이미 설치된 경우 스킵됨)
!pip install -q datasets jsonlines tqdm

print("✅ 파인튜닝 라이브러리 설치 완료!")

# 설치 검증
print("🧪 bitsandbytes 검증 중...")
try:
    import bitsandbytes as bnb
    import torch
    print(f"✅ bitsandbytes 버전: {bnb.__version__}")
    
    if torch.cuda.is_available():
        print(f"✅ CUDA 사용 가능: {torch.cuda.get_device_name(0)}")
        # 간단한 CUDA 테스트
        test_tensor = torch.randn(10, 10).cuda()
        print("✅ CUDA 텐서 생성 성공")
    else:
        print("⚠️ CUDA를 사용할 수 없습니다")

    # 허깅페이스 Hub 확인
    import huggingface_hub
    print(f"✅ huggingface_hub 버전: {huggingface_hub.__version__}")
        
    print("🎉 파인튜닝 환경 준비 완료!")
    
except Exception as e:
    print(f"❌ 검증 실패: {e}")
    print("💡 해결 방법:")
    print("1. 런타임 > 런타임 다시 시작")
    print("2. 셀을 다시 실행해보세요")

print("\n" + "="*60)
print("📝 참고: 01, 02번 노트북에서 이미 설치된 라이브러리는 재사용됩니다")
print("🔄 다음 셀부터 파인튜닝 작업을 시작합니다")
print("="*60)

In [ ]:
import os
import json
import jsonlines
import torch
import numpy as np
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from typing import Dict, List, Any, Optional
import warnings
from tqdm import tqdm

# Transformers 관련
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    TrainerCallback,
    DataCollatorForLanguageModeling
)

# PEFT 관련
from peft import (
    LoraConfig,
    get_peft_model,
    prepare_model_for_kbit_training,
    TaskType,
    PeftModel
)

# Dataset
from datasets import Dataset

# 허깅페이스 Hub
from huggingface_hub import HfApi, login, create_repo

warnings.filterwarnings('ignore')

# 한글 폰트 설정 (matplotlib) - 학습 결과 분석 차트에서 한글이 깨지지 않도록 설정
# 파인튜닝 과정의 Loss 곡선, GPU 메모리 사용량 등의 차트에서 한글 표시를 위해 필요
print("🔧 한글 폰트 설정 중...")
!apt-get update -qq
!apt-get install fonts-nanum -qq > /dev/null

import matplotlib.font_manager as fm

# 나눔바른고딕 폰트 경로 설정
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
# 폰트 매니저에 폰트 추가 - 학습 분석 그래프에서 한글 표시를 위해 필요
fm.fontManager.addfont(fontpath)

# matplotlib 설정 업데이트 - 학습 과정 시각화에서 한글이 정상적으로 표시됨
plt.rcParams.update({
    'font.family': 'NanumBarunGothic',  # 기본 폰트를 나눔바른고딕으로 설정
    'axes.unicode_minus': False         # 음수 기호 표시 문제 해결 (Loss 값 표시에서 중요)
})

print("✅ 한글 폰트 설정 완료 - 학습 결과 차트에서 한글이 정상 표시됩니다")

# GPU 정보 확인 - 파인튜닝에 필요한 하드웨어 환경 점검
print(f"🔧 파인튜닝 환경 정보:")
print(f"PyTorch 버전: {torch.__version__}")
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU 개수: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        gpu_name = torch.cuda.get_device_name(i)
        gpu_memory = torch.cuda.get_device_properties(i).total_memory / 1024**3
        print(f"GPU {i}: {gpu_name}")
        print(f"  전체 메모리: {gpu_memory:.1f} GB")
        
        # 현재 메모리 사용량도 표시
        allocated = torch.cuda.memory_allocated(i) / 1024**3
        print(f"  현재 사용 중: {allocated:.1f} GB")
else:
    print("⚠️ CUDA를 사용할 수 없습니다. CPU에서 실행됩니다 (매우 느림).")
    print("   GPU 환경에서 실행하는 것을 권장합니다.")

print("\n✅ 라이브러리 import 및 환경 설정 완료!")

## 2. 모델 및 토크나이저 로드

### 🤖 EXAONE-3.5-2.4B-Instruct 모델
- **개발사**: LG AI Research
- **크기**: 2.4B 파라미터
- **특징**: 한국어 특화, Instruction Following 최적화
- **장점**: Colab 무료 환경에서 실행 가능

In [ ]:
# 모델 설정
MODEL_NAME = "LGAI-EXAONE/EXAONE-3.5-2.4B-Instruct"
MAX_LENGTH = 4096  # 최대 토큰 길이

print(f"🔄 모델 로드 중: {MODEL_NAME}")

# 토크나이저 로드
print("📝 토크나이저 로드...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)

# 패딩 토큰 설정 (없는 경우)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    print("✅ 패딩 토큰을 EOS 토큰으로 설정")

print(f"✅ 토크나이저 로드 완료")
print(f"  어휘 크기: {tokenizer.vocab_size:,}")
print(f"  패딩 토큰: '{tokenizer.pad_token}' (ID: {tokenizer.pad_token_id})")
print(f"  EOS 토큰: '{tokenizer.eos_token}' (ID: {tokenizer.eos_token_id})")

## 3. QLoRA 설정

### ⚙️ 4-bit 양자화 설정
- **nf4**: 4-bit NormalFloat 데이터 타입 사용
- **double_quant**: 양자화 상수도 양자화 (더 많은 메모리 절약)
- **compute_dtype**: 연산 시 사용할 데이터 타입

In [ ]:
# 4-bit 양자화 설정
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,                    # 4-bit 양자화 활성화
    bnb_4bit_use_double_quant=True,       # 양자화 상수도 양자화 (더 많은 메모리 절약)
    bnb_4bit_quant_type="nf4",           # NormalFloat4 양자화 방식
    bnb_4bit_compute_dtype=torch.bfloat16, # 연산용 데이터 타입
)

print("⚙️ 4-bit 양자화 설정 완료:")
print(f"  양자화 방식: NF4")
print(f"  Double quantization: 활성화")
print(f"  연산 데이터 타입: bfloat16")

# LoRA 설정
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,         # 언어 모델링 태스크
    r=8,                                  # LoRA rank (낮을수록 파라미터 적음)
    lora_alpha=16,                        # LoRA scaling factor (보통 r의 2배)
    lora_dropout=0.05,                    # LoRA 드롭아웃
    target_modules=[                      # LoRA를 적용할 모듈들
        "q_proj",    # Query projection
        "k_proj",    # Key projection  
        "v_proj",    # Value projection
        "out_proj"   # Output projection
    ],
    bias="none",                         # 바이어스는 학습하지 않음
)

print("\n🎯 LoRA 설정 완료:")
print(f"  Rank (r): {lora_config.r}")
print(f"  Alpha: {lora_config.lora_alpha}")
print(f"  Dropout: {lora_config.lora_dropout}")
print(f"  대상 모듈: {lora_config.target_modules}")

# 학습 가능한 파라미터 비율 추정
total_params_estimate = 2.4e9  # 2.4B 파라미터
lora_params_estimate = len(lora_config.target_modules) * lora_config.r * 2560 * 2  # 대략적 추정
lora_ratio = lora_params_estimate / total_params_estimate

print(f"\n📊 예상 학습 파라미터 비율:")
print(f"  전체 파라미터: ~{total_params_estimate/1e9:.1f}B")
print(f"  LoRA 파라미터: ~{lora_params_estimate/1e6:.1f}M")
print(f"  학습 비율: ~{lora_ratio:.3%}")
print(f"  메모리 절약: ~75% (4-bit 양자화)")

## 4. 모델 로드 및 PEFT 적용

In [ ]:
def load_model_with_qlora(model_name: str, bnb_config: BitsAndBytesConfig, 
                         lora_config: LoraConfig) -> tuple:
    """
    QLoRA가 적용된 모델을 로드하는 함수
    
    Returns:
        (model, trainable_params, total_params)
    """
    print("🔄 모델 로드 중... (시간이 걸릴 수 있습니다)")
    
    # 모델 로드 (4-bit 양자화 적용)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        quantization_config=bnb_config,
        device_map="auto",              # 자동으로 GPU에 배치
        trust_remote_code=True,
        torch_dtype=torch.bfloat16,
        # flash_attention_2는 일부 환경에서 문제가 될 수 있으므로 제거
        # attn_implementation="flash_attention_2" if torch.cuda.is_available() else "eager"
    )
    
    print("✅ 기본 모델 로드 완료")
    
    # Gradient checkpointing 활성화 (메모리 절약)
    model.gradient_checkpointing_enable()
    
    # k-bit training 준비
    model = prepare_model_for_kbit_training(model)
    
    # LoRA 적용
    print("🔄 LoRA 적용 중...")
    model = get_peft_model(model, lora_config)
    
    # 학습 가능한 파라미터 수 계산
    trainable_params = 0
    total_params = 0
    
    for name, param in model.named_parameters():
        total_params += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    
    print("✅ LoRA 적용 완료")
    
    return model, trainable_params, total_params

# 모델 설정 및 LoRA 설정은 이전 셀에서 정의된 변수(MODEL_NAME, bnb_config, lora_config)를 사용합니다.
if 'bnb_config' in globals() and 'lora_config' in globals():
    model, trainable_params, total_params = load_model_with_qlora(
        MODEL_NAME, bnb_config, lora_config
    )

    # 파라미터 정보 출력
    print(f"\n📊 모델 파라미터 정보:")
    print(f"  총 파라미터: {total_params:,} ({total_params/1e9:.2f}B)")
    print(f"  학습 가능한 파라미터: {trainable_params:,} ({trainable_params/1e6:.2f}M)")
    print(f"  학습 비율: {100 * trainable_params / total_params:.3f}%")

    # GPU 메모리 사용량 확인
    if torch.cuda.is_available():
        print(f"\n💾 GPU 메모리 사용량:")
        for i in range(torch.cuda.device_count()):
            allocated = torch.cuda.memory_allocated(i) / 1024**3
            cached = torch.cuda.memory_reserved(i) / 1024**3
            total_memory = torch.cuda.get_device_properties(i).total_memory / 1024**3
            print(f"  GPU {i}: {allocated:.1f}GB / {total_memory:.1f}GB (할당됨)")
            print(f"          {cached:.1f}GB / {total_memory:.1f}GB (예약됨)")
else:
    print("❌ bnb_config 또는 lora_config가 정의되지 않았습니다.")
    print("💡 이전 셀들을 먼저 실행해주세요.")

## 5. 데이터셋 로드 및 전처리

### 📁 전처리된 RAFT 데이터셋 활용
- 이전 실습에서 생성한 한국어 RAFT 데이터셋 사용
- Chat template 적용된 메시지 형태
- Train/Valid 분할 완료

In [ ]:
def load_training_data() -> tuple:
    """
    전처리된 학습 데이터를 로드하는 함수
    
    Returns:
        (train_dataset, valid_dataset, metadata)
    """
    print("📂 학습 데이터 로드 중...")
    
    try:
        # Train 데이터 로드
        train_data = []
        with jsonlines.open("processed_data/train_raft_ko.jsonl", "r") as reader:
            train_data = list(reader)
        
        # Valid 데이터 로드
        valid_data = []
        with jsonlines.open("processed_data/valid_raft_ko.jsonl", "r") as reader:
            valid_data = list(reader)
        
        # 메타데이터 로드
        with open("processed_data/metadata.json", "r", encoding="utf-8") as f:
            metadata = json.load(f)
        
        print(f"✅ 데이터 로드 완료:")
        print(f"  Train: {len(train_data)}개 샘플")
        print(f"  Valid: {len(valid_data)}개 샘플")
        
        return train_data, valid_data, metadata
        
    except FileNotFoundError as e:
        print(f"❌ 파일을 찾을 수 없습니다: {e}")
        print("💡 먼저 01_data_preprocessing_and_validation.ipynb를 실행하세요.")
        return None, None, None

def format_chat_template(messages: List[Dict[str, str]]) -> str:
    """
    메시지를 EXAONE 채팅 템플릿 형태로 변환
    
    Args:
        messages: 채팅 메시지 리스트
        
    Returns:
        포맷팅된 텍스트
    """
    return tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=False  # Assistant 응답까지 포함
    )

def create_training_dataset(data: List[Dict], max_samples: Optional[int] = None) -> Dataset:
    """
    학습용 데이터셋 생성 - Trainer와 호환되는 형식으로 생성
    
    Args:
        data: 원본 데이터
        max_samples: 최대 샘플 수 (None이면 전체 사용)
        
    Returns:
        Hugging Face Dataset
    """
    print(f"🔄 학습 데이터셋 생성 중...")
    
    # 샘플 수 제한
    if max_samples is not None and len(data) > max_samples:
        data = data[:max_samples]
        print(f"  샘플 수 제한: {max_samples}개")
    
    texts = []
    
    for item in tqdm(data, desc="데이터 변환"):
        if "messages" not in item:
            continue
            
        # 채팅 템플릿 적용
        formatted_text = format_chat_template(item["messages"])
        texts.append(formatted_text)
    
    print(f"✅ 텍스트 변환 완료: {len(texts)}개")
    
    # 토크나이징을 여기서 수행하지 않고, 텍스트만 저장
    # Trainer가 data_collator를 통해 배치별로 토크나이징 수행
    dataset_dict = {
        "text": texts
    }
    
    dataset = Dataset.from_dict(dataset_dict)
    print(f"✅ 데이터셋 생성 완료: {len(dataset)}개 샘플")
    
    return dataset

# 데이터 로드
train_data, valid_data, metadata = load_training_data()

if train_data is not None:
    # Colab 무료 환경 고려하여 샘플 수 제한 (필요시 조정)
    MAX_TRAIN_SAMPLES = 400  # 학습 시간과 메모리를 고려한 제한
    MAX_VALID_SAMPLES = 100
    
    # 데이터셋 생성
    train_dataset = create_training_dataset(train_data, MAX_TRAIN_SAMPLES)
    valid_dataset = create_training_dataset(valid_data, MAX_VALID_SAMPLES)
    
    # 데이터셋 정보 출력
    print(f"\n📊 최종 데이터셋 정보:")
    print(f"  Train 샘플: {len(train_dataset)}개")
    print(f"  Valid 샘플: {len(valid_dataset)}개")
    
    # 샘플 데이터 미리보기
    print(f"\n📋 샘플 데이터:")
    if len(train_dataset) > 0:
        sample_text = train_dataset[0]["text"]
        print(f"  텍스트 길이: {len(sample_text)} 문자")
        print(f"  텍스트 미리보기: {sample_text[:200]}...")
        
        # 토큰 길이 확인
        tokens = tokenizer.encode(sample_text)
        print(f"  토큰 길이: {len(tokens)} 토큰")

else:
    print("❌ 데이터를 로드할 수 없어 학습을 진행할 수 없습니다.")

## 6. GPU 메모리 모니터링 콜백

### 🔍 메모리 모니터링의 중요성
- Colab 환경에서 GPU 메모리는 제한적 (15GB)
- OOM(Out of Memory) 오류를 사전에 방지
- 학습 중 메모리 사용량 실시간 추적

In [ ]:
class GPUMemoryCallback(TrainerCallback):
    """
    GPU 메모리 사용량을 모니터링하는 콜백 클래스
    """
    
    def __init__(self, log_every_n_steps: int = 10, memory_threshold: float = 0.9):
        """
        Args:
            log_every_n_steps: 로그 출력 주기
            memory_threshold: 메모리 사용량 경고 임계치 (90%)
        """
        self.log_every_n_steps = log_every_n_steps
        self.memory_threshold = memory_threshold
        self.memory_history = []
        
    def on_step_end(self, args, state, control, **kwargs):
        """
        각 스텝 종료 후 호출되는 메서드
        """
        if not torch.cuda.is_available():
            return
            
        if state.global_step % self.log_every_n_steps == 0:
            # 메모리 정보 수집
            memory_info = self._collect_memory_info()
            self.memory_history.append({
                "step": state.global_step,
                "memory_info": memory_info
            })
            
            # 로그 출력
            self._log_memory_info(state.global_step, memory_info)
            
            # 메모리 경고
            for i, info in enumerate(memory_info):
                if info["usage_ratio"] > self.memory_threshold:
                    print(f"⚠️ GPU {i} 메모리 사용량 경고: {info['usage_ratio']:.1%}")
    
    def _collect_memory_info(self) -> List[Dict[str, float]]:
        """
        GPU 메모리 정보 수집
        """
        memory_info = []
        
        for i in range(torch.cuda.device_count()):
            allocated = torch.cuda.memory_allocated(i) / 1024**3
            reserved = torch.cuda.memory_reserved(i) / 1024**3
            total = torch.cuda.get_device_properties(i).total_memory / 1024**3
            
            memory_info.append({
                "gpu_id": i,
                "allocated_gb": allocated,
                "reserved_gb": reserved,
                "total_gb": total,
                "usage_ratio": reserved / total if total > 0 else 0
            })
        
        return memory_info
    
    def _log_memory_info(self, step: int, memory_info: List[Dict[str, float]]):
        """
        메모리 정보 로그 출력
        """
        print(f"\n📊 Step {step} - GPU 메모리 상태:")
        for info in memory_info:
            gpu_id = info["gpu_id"]
            allocated = info["allocated_gb"]
            reserved = info["reserved_gb"]
            total = info["total_gb"]
            usage = info["usage_ratio"]
            
            print(f"  GPU {gpu_id}: {reserved:.1f}GB/{total:.1f}GB ({usage:.1%}) "
                  f"[할당됨: {allocated:.1f}GB]")
    
    def get_memory_history(self) -> List[Dict]:
        """
        메모리 사용 히스토리 반환
        """
        return self.memory_history

# 콜백 인스턴스 생성
memory_callback = GPUMemoryCallback(log_every_n_steps=5, memory_threshold=0.85)

print("🔍 GPU 메모리 모니터링 콜백 생성 완료")
if torch.cuda.is_available():
    print(f"  모니터링 주기: 5 스텝마다")
    print(f"  경고 임계치: 85%")
else:
    print("  CPU 모드에서는 메모리 모니터링이 비활성화됩니다.")

## 7. 학습 설정

### 🎯 Colab 무료 환경 최적화 설정
- **배치 크기**: 1 (메모리 절약)
- **Gradient Accumulation**: 32 (실질적 배치 크기 = 32)
- **Learning Rate**: 1e-4 (LoRA에 적합한 값)
- **FP16**: 메모리와 속도 최적화

In [ ]:
# 학습 설정
def create_training_arguments() -> TrainingArguments:
    """
    학습 파라미터 설정
    """
    
    # 출력 디렉토리 설정
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_dir = f"./fine_tuned_model_{timestamp}"
    
    return TrainingArguments(
        # 📁 디렉토리 설정
        output_dir=output_dir,
        logging_dir=f"{output_dir}/logs",
        
        # 🏋️ 학습 설정
        num_train_epochs=1,                    # 에포크 수 (시간 고려하여 1)
        per_device_train_batch_size=1,         # GPU당 배치 크기
        per_device_eval_batch_size=1,          # 평가 배치 크기
        gradient_accumulation_steps=32,        # 그래디언트 누적 (실질 배치=32)
        
        # 📈 최적화 설정
        learning_rate=1e-4,                   # 학습률
        warmup_steps=50,                      # 웜업 스텝
        optim="paged_adamw_8bit",            # 8bit AdamW 옵티마이저
        
        # 💾 메모리 최적화
        fp16=True,                           # FP16 정밀도
        dataloader_pin_memory=False,         # 메모리 핀 비활성화
        gradient_checkpointing=True,         # 그래디언트 체크포인팅
        
        # 📊 로깅 및 평가
        logging_steps=5,                     # 로그 출력 주기
        eval_steps=50,                       # 평가 주기
        eval_strategy="steps",               # 스텝 기반 평가 (evaluation_strategy → eval_strategy)
        
        # 💾 저장 설정
        save_steps=100,                      # 체크포인트 저장 주기
        save_total_limit=2,                  # 최대 체크포인트 개수
        save_strategy="steps",               # 스텝 기반 저장
        
        # 🎯 기타 설정
        remove_unused_columns=False,         # 사용하지 않는 컬럼 제거 안함
        report_to="none",                   # wandb 등 리포트 비활성화
        load_best_model_at_end=True,         # 최고 성능 모델 로드
        metric_for_best_model="eval_loss",   # 최고 모델 선택 기준
        greater_is_better=False,             # Loss는 낮을수록 좋음
        
        # 🔄 재현성
        seed=42,
        data_seed=42,
    )

# 학습 설정 생성
training_args = create_training_arguments()

print("⚙️ 학습 설정 완료:")
print(f"  출력 디렉토리: {training_args.output_dir}")
print(f"  에포크 수: {training_args.num_train_epochs}")
print(f"  배치 크기: {training_args.per_device_train_batch_size}")
print(f"  그래디언트 누적: {training_args.gradient_accumulation_steps}")
print(f"  실질적 배치 크기: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"  학습률: {training_args.learning_rate}")
print(f"  FP16: {training_args.fp16}")
print(f"  옵티마이저: {training_args.optim}")

# 예상 학습 시간 계산
if train_dataset is not None:
    total_steps = len(train_dataset) // (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps) * training_args.num_train_epochs
    estimated_time_minutes = total_steps * 0.5  # 스텝당 약 30초 추정
    
    print(f"\n⏱️ 예상 학습 시간:")
    print(f"  총 스텝: {total_steps}")
    print(f"  예상 시간: {estimated_time_minutes:.0f}분 ({estimated_time_minutes/60:.1f}시간)")

## 8. 데이터 콜레이터 설정

### 📦 Dynamic Padding
- 배치마다 최적 길이로 패딩 (메모리 효율)
- Language Modeling용 라벨 자동 생성

In [ ]:
# 데이터 콜레이터 설정 - 텍스트를 토크나이징하고 배치 처리
def tokenize_function(examples):
    """
    텍스트를 토크나이징하는 함수
    """
    # 텍스트를 토크나이징
    tokenized = tokenizer(
        examples["text"],
        truncation=True,
        padding=False,  # 배치 시 동적 패딩
        max_length=MAX_LENGTH,
        return_tensors="pt" if len(examples["text"]) == 1 else None
    )
    
    # labels는 input_ids와 동일 (language modeling)
    tokenized["labels"] = tokenized["input_ids"].copy()
    
    return tokenized

# 데이터셋에 토크나이징 적용
if 'train_dataset' in locals() and train_dataset is not None:
    print("🔄 데이터셋 토크나이징 중...")
    
    # 토크나이징 적용
    train_dataset = train_dataset.map(
        tokenize_function,
        batched=True,
        remove_columns=["text"],  # 원본 텍스트 제거
        desc="토크나이징 진행"
    )
    
    valid_dataset = valid_dataset.map(
        tokenize_function,
        batched=True,
        remove_columns=["text"],  # 원본 텍스트 제거
        desc="토크나이징 진행"
    )
    
    print("✅ 토크나이징 완료")

# 데이터 콜레이터 설정
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,  # Causal Language Modeling (MLM 아님)
    pad_to_multiple_of=8,  # 텐서 연산 최적화를 위해 8의 배수로 패딩
)

print("📦 데이터 콜레이터 설정 완료:")
print(f"  MLM: {data_collator.mlm}")
print(f"  패딩 단위: {data_collator.pad_to_multiple_of}")
print(f"  패딩 토큰 ID: {tokenizer.pad_token_id}")

# 샘플 데이터 확인
if 'train_dataset' in locals() and len(train_dataset) > 0:
    print(f"\n📋 토크나이징된 샘플 확인:")
    sample = train_dataset[0]
    print(f"  Input IDs 길이: {len(sample['input_ids'])}")
    print(f"  Labels 길이: {len(sample['labels'])}")
    print(f"  Attention Mask 길이: {len(sample['attention_mask'])}")

## 9. Trainer 설정 및 학습 시작

### 🚀 파인튜닝 실행
- 실시간 GPU 메모리 모니터링
- 자동 체크포인트 저장
- 평가 메트릭 추적

In [ ]:
# 학습 전 사전 점검
def pre_training_check():
    """
    학습 시작 전 환경 점검
    """
    print("🔍 학습 전 환경 점검:")
    
    # GPU 메모리 확인
    if torch.cuda.is_available():
        for i in range(torch.cuda.device_count()):
            total_memory = torch.cuda.get_device_properties(i).total_memory / 1024**3
            allocated = torch.cuda.memory_allocated(i) / 1024**3
            print(f"  GPU {i}: {allocated:.1f}GB/{total_memory:.1f}GB 사용 중")
            
            if allocated / total_memory > 0.8:
                print(f"  ⚠️ GPU {i} 메모리 사용량이 높습니다. 학습 중 OOM 위험이 있습니다.")
    
    # 모델 상태 확인
    if model.training:
        print("  ✅ 모델이 학습 모드입니다.")
    else:
        print("  ⚠️ 모델이 평가 모드입니다. 학습 모드로 전환됩니다.")
        model.train()
    
    # 데이터셋 크기 확인
    print(f"  데이터: Train {len(train_dataset)}개, Valid {len(valid_dataset)}개")
    
    return True

# Trainer 생성
if train_dataset is not None and len(train_dataset) > 0:
    print("🔄 Trainer 생성 중...")
    
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=valid_dataset if len(valid_dataset) > 0 else None,
        data_collator=data_collator,
        callbacks=[memory_callback],  # GPU 메모리 모니터링 콜백
    )
    
    print("✅ Trainer 생성 완료")
    
    # 학습 전 점검
    if pre_training_check():
        print("\n🚀 파인튜닝 시작!")
        print("=" * 50)
        
        # 학습 시작 시간 기록
        training_start_time = datetime.now()
        
        try:
            # 실제 학습 실행
            training_result = trainer.train()
            
            # 학습 완료 시간 기록
            training_end_time = datetime.now()
            training_duration = training_end_time - training_start_time
            
            print("\n🎉 파인튜닝 완료!")
            print("=" * 50)
            print(f"학습 시간: {training_duration}")
            print(f"최종 Loss: {training_result.training_loss:.4f}")
            
            # 학습 결과 저장
            training_summary = {
                "model_name": MODEL_NAME,
                "training_start": training_start_time.isoformat(),
                "training_end": training_end_time.isoformat(),
                "training_duration": str(training_duration),
                "final_loss": training_result.training_loss,
                "total_steps": training_result.global_step,
                "trainable_parameters": trainable_params,
                "total_parameters": total_params,
                "training_samples": len(train_dataset),
                "validation_samples": len(valid_dataset)
            }
            
            # 결과 저장
            with open(f"{training_args.output_dir}/training_summary.json", "w", encoding="utf-8") as f:
                json.dump(training_summary, f, ensure_ascii=False, indent=2, default=str)
            
            print(f"\n💾 학습 결과 저장: {training_args.output_dir}/training_summary.json")
            
        except KeyboardInterrupt:
            print("\n⏹️ 사용자에 의해 학습이 중단되었습니다.")
            
        except RuntimeError as e:
            if "out of memory" in str(e).lower():
                print("\n❌ GPU 메모리 부족으로 학습이 중단되었습니다.")
                print("💡 해결 방법:")
                print("   1. 배치 크기를 더 줄여보세요 (per_device_train_batch_size=1)")
                print("   2. 그래디언트 누적 단계를 늘려보세요 (gradient_accumulation_steps=64)")
                print("   3. 최대 토큰 길이를 줄여보세요 (MAX_LENGTH=2048)")
                print("   4. 학습 샘플 수를 줄여보세요 (MAX_TRAIN_SAMPLES=200)")
            else:
                print(f"\n❌ 학습 중 오류 발생: {e}")
            
        except Exception as e:
            print(f"\n❌ 예상치 못한 오류 발생: {e}")
            
else:
    print("❌ 학습 데이터가 없어 파인튜닝을 진행할 수 없습니다.")
    print("💡 먼저 01_data_preprocessing_and_validation.ipynb를 실행하세요.")

## 10. 학습 결과 분석 및 시각화

In [ ]:
def analyze_training_results(trainer: Trainer, memory_callback: GPUMemoryCallback):
    """
    학습 결과 분석 및 시각화
    """
    print("📊 학습 결과 분석 중...")
    
    # 1. 학습 로그 분석
    log_history = trainer.state.log_history
    
    if not log_history:
        print("⚠️ 학습 로그가 없습니다.")
        return
    
    # 학습 및 평가 로그 분리
    train_logs = [log for log in log_history if 'loss' in log and 'eval_loss' not in log]
    eval_logs = [log for log in log_history if 'eval_loss' in log]
    
    print(f"\n📈 학습 진행 상황:")
    print(f"  총 학습 스텝: {len(train_logs)}")
    print(f"  평가 횟수: {len(eval_logs)}")
    
    if train_logs:
        initial_loss = train_logs[0].get('loss', 0)
        final_loss = train_logs[-1].get('loss', 0)
        loss_improvement = initial_loss - final_loss
        
        print(f"  초기 Loss: {initial_loss:.4f}")
        print(f"  최종 Loss: {final_loss:.4f}")
        print(f"  Loss 개선: {loss_improvement:.4f} ({loss_improvement/initial_loss:.1%})")
    
    # 2. 시각화
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # 2-1. Training Loss 곡선
    if train_logs:
        steps = [log.get('step', 0) for log in train_logs]
        losses = [log.get('loss', 0) for log in train_logs]
        
        axes[0, 0].plot(steps, losses, 'b-', label='Training Loss', linewidth=2)
        axes[0, 0].set_xlabel('Steps')
        axes[0, 0].set_ylabel('Loss')
        axes[0, 0].set_title('Training Loss Curve')
        axes[0, 0].grid(True, alpha=0.3)
        axes[0, 0].legend()
    
    # 2-2. Evaluation Loss 곡선
    if eval_logs:
        eval_steps = [log.get('step', 0) for log in eval_logs]
        eval_losses = [log.get('eval_loss', 0) for log in eval_logs]
        
        axes[0, 1].plot(eval_steps, eval_losses, 'r-', label='Validation Loss', linewidth=2)
        axes[0, 1].set_xlabel('Steps')
        axes[0, 1].set_ylabel('Loss')
        axes[0, 1].set_title('Validation Loss Curve')
        axes[0, 1].grid(True, alpha=0.3)
        axes[0, 1].legend()
    
    # 2-3. GPU 메모리 사용량
    memory_history = memory_callback.get_memory_history()
    if memory_history:
        memory_steps = [item['step'] for item in memory_history]
        memory_usage = [item['memory_info'][0]['usage_ratio'] * 100 
                       for item in memory_history if item['memory_info']]
        
        if memory_usage:
            axes[1, 0].plot(memory_steps, memory_usage, 'g-', label='GPU Memory Usage', linewidth=2)
            axes[1, 0].axhline(y=85, color='orange', linestyle='--', label='Warning (85%)')
            axes[1, 0].axhline(y=95, color='red', linestyle='--', label='Critical (95%)')
            axes[1, 0].set_xlabel('Steps')
            axes[1, 0].set_ylabel('Memory Usage (%)')
            axes[1, 0].set_title('GPU Memory Usage')
            axes[1, 0].grid(True, alpha=0.3)
            axes[1, 0].legend()
    
    # 2-4. Learning Rate 스케줄
    lr_logs = [log.get('learning_rate', 0) for log in train_logs if 'learning_rate' in log]
    if lr_logs:
        lr_steps = [log.get('step', 0) for log in train_logs if 'learning_rate' in log]
        
        axes[1, 1].plot(lr_steps, lr_logs, 'm-', label='Learning Rate', linewidth=2)
        axes[1, 1].set_xlabel('Steps')
        axes[1, 1].set_ylabel('Learning Rate')
        axes[1, 1].set_title('Learning Rate Schedule')
        axes[1, 1].grid(True, alpha=0.3)
        axes[1, 1].legend()
    
    plt.tight_layout()
    
    # 그래프 저장
    output_dir = trainer.args.output_dir
    plt.savefig(f"{output_dir}/training_analysis.png", dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"\n💾 학습 분석 그래프 저장: {output_dir}/training_analysis.png")
    
    # 3. 상세 분석 리포트 생성
    analysis_report = {
        "training_summary": {
            "total_steps": len(train_logs),
            "evaluation_count": len(eval_logs),
            "initial_loss": train_logs[0].get('loss', 0) if train_logs else 0,
            "final_loss": train_logs[-1].get('loss', 0) if train_logs else 0,
            "loss_improvement": loss_improvement if train_logs else 0,
            "improvement_percentage": (loss_improvement/initial_loss*100) if train_logs and initial_loss > 0 else 0
        },
        "memory_analysis": {
            "max_memory_usage": max(memory_usage) if memory_usage else 0,
            "avg_memory_usage": np.mean(memory_usage) if memory_usage else 0,
            "memory_warnings": sum(1 for usage in memory_usage if usage > 85) if memory_usage else 0
        },
        "recommendations": []
    }
    
    # 권장사항 생성
    if analysis_report["training_summary"]["improvement_percentage"] < 5:
        analysis_report["recommendations"].append("Loss 개선이 미미합니다. 더 많은 에포크나 다른 학습률을 시도해보세요.")
    
    if analysis_report["memory_analysis"]["max_memory_usage"] > 90:
        analysis_report["recommendations"].append("GPU 메모리 사용량이 높습니다. 배치 크기를 줄이는 것을 고려하세요.")
    
    if not analysis_report["recommendations"]:
        analysis_report["recommendations"].append("학습이 안정적으로 진행되었습니다!")
    
    # 리포트 저장
    with open(f"{output_dir}/analysis_report.json", "w", encoding="utf-8") as f:
        json.dump(analysis_report, f, ensure_ascii=False, indent=2)
    
    print(f"💾 분석 리포트 저장: {output_dir}/analysis_report.json")
    
    return analysis_report

# 학습 완료 후 결과 분석 실행
if 'trainer' in locals() and 'training_result' in locals():
    analysis_report = analyze_training_results(trainer, memory_callback)
    
    # 분석 결과 요약 출력
    print("\n📋 학습 결과 요약:")
    print(f"  Loss 개선: {analysis_report['training_summary']['improvement_percentage']:.1f}%")
    print(f"  최대 메모리 사용: {analysis_report['memory_analysis']['max_memory_usage']:.1f}%")
    print(f"\n💡 권장사항:")
    for rec in analysis_report['recommendations']:
        print(f"   - {rec}")
else:
    print("⚠️ 학습이 완료되지 않아 결과 분석을 수행할 수 없습니다.")

## 11. 모델 저장 및 테스트

### 💾 학습된 모델 저장
- LoRA 어댑터만 저장 (용량 효율적)
- 추후 쉬운 로드를 위한 설정 파일 포함

In [ ]:
def save_fine_tuned_model(trainer: Trainer, output_path: str = None) -> str:
    """
    파인튜닝된 모델 저장
    
    Args:
        trainer: 학습된 Trainer 객체
        output_path: 저장 경로 (None이면 자동 생성)
        
    Returns:
        저장된 모델 경로
    """
    if output_path is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        output_path = f"./exaone_raft_lora_{timestamp}"
    
    print(f"💾 모델 저장 중: {output_path}")
    
    # 모델 저장 (LoRA 어댑터만)
    trainer.model.save_pretrained(output_path)
    
    # 토크나이저도 함께 저장
    tokenizer.save_pretrained(output_path)
    
    # 모델 정보 저장 (JSON 직렬화 가능한 형태로 변환)
    model_info = {
        "base_model": MODEL_NAME,
        "model_type": "QLoRA",
        "task_type": "RAG Fine-tuning",
        "save_timestamp": datetime.now().isoformat(),
        "lora_config": {
            "r": lora_config.r,
            "lora_alpha": lora_config.lora_alpha,
            "lora_dropout": lora_config.lora_dropout,
            "target_modules": list(lora_config.target_modules)  # set을 list로 변환
        },
        "training_info": {
            "trainable_parameters": trainable_params,
            "total_parameters": total_params,
            "training_samples": len(train_dataset) if train_dataset else 0
        }
    }
    
    with open(f"{output_path}/model_info.json", "w", encoding="utf-8") as f:
        json.dump(model_info, f, ensure_ascii=False, indent=2)
    
    print(f"✅ 모델 저장 완료: {output_path}")
    print(f"   - LoRA 어댑터: adapter_model.safetensors")
    print(f"   - 설정 파일: adapter_config.json")
    print(f"   - 토크나이저: tokenizer.json, tokenizer_config.json")
    print(f"   - 모델 정보: model_info.json")
    
    return output_path

def upload_to_huggingface(model_path: str, repo_name: str = "ryanu/my-exaone-raft-model", private: bool = False):
    """
    파인튜닝된 모델을 허깅페이스 Hub에 업로드
    
    Args:
        model_path: 로컬 모델 경로
        repo_name: 허깅페이스 리포지토리 이름 (기본: ryanu/my-exaone-raft-model)
        private: 프라이빗 리포지토리 여부
    """
    
    print("\n🚀 허깅페이스 Hub 업로드 준비 중...")
    
    # 허깅페이스 로그인 확인
    try:
        api = HfApi()
        user_info = api.whoami()
        username = user_info["name"]
        print(f"✅ 허깅페이스 로그인 확인: {username}")
        
    except Exception as e:
        print("❌ 허깅페이스 로그인이 필요합니다!")
        print("💡 다음 단계를 따라주세요:")
        print("1. https://huggingface.co/settings/tokens 에서 토큰 생성")
        print("2. 아래 코드 실행:")
        print("   from huggingface_hub import login")
        print("   login()  # 토큰 입력")
        print("3. 또는 환경변수 설정:")
        print("   import os")
        print("   os.environ['HF_TOKEN'] = 'your_token_here'")
        return None
    
    try:
        print(f"📦 리포지토리 생성 중: {repo_name}")
        
        # 리포지토리 생성 (이미 존재하면 무시)
        try:
            create_repo(
                repo_id=repo_name,
                private=private,
                exist_ok=True,
                repo_type="model"
            )
            print(f"✅ 리포지토리 생성 완료: {repo_name}")
        except Exception as e:
            if "already exists" in str(e):
                print(f"✅ 기존 리포지토리 사용: {repo_name}")
            else:
                raise e
        
        # 모델 카드 생성
        model_card_content = f"""---
license: apache-2.0
base_model: {MODEL_NAME}
tags:
- peft
- lora
- korean
- rag
- exaone
language:
- ko
---

# EXAONE RAG Fine-tuned Model with LoRA

이 모델은 EXAONE-3.5-2.4B-Instruct를 기반으로 한국어 RAG 데이터셋으로 파인튜닝된 모델입니다.

## Model Details

- **Base Model**: {MODEL_NAME}
- **Fine-tuning Method**: QLoRA (4-bit quantization + LoRA)
- **Task**: Retrieval-Augmented Generation (RAG)
- **Language**: Korean
- **Training Data**: RAFT methodology based Korean RAG dataset

## Usage

```python
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel

# 베이스 모델과 토크나이저 로드
base_model = AutoModelForCausalLM.from_pretrained("{MODEL_NAME}")
tokenizer = AutoTokenizer.from_pretrained("{MODEL_NAME}")

# LoRA 어댑터 적용
model = PeftModel.from_pretrained(base_model, "{repo_name}")

# 추론 예시
messages = [
    {{"role": "system", "content": "주어진 컨텍스트를 바탕으로 질문에 답변하세요."}},
    {{"role": "user", "content": "컨텍스트: 한국의 수도는 서울입니다. 질문: 한국의 수도는?""}}
]

input_text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
inputs = tokenizer.encode(input_text, return_tensors="pt")

with torch.no_grad():
    outputs = model.generate(inputs, max_new_tokens=100, temperature=0.7)
    
response = tokenizer.decode(outputs[0][inputs.shape[1]:], skip_special_tokens=True)
print(response)
```

## Training Details

- **Training Framework**: Hugging Face Transformers + PEFT
- **Optimization**: 8-bit AdamW
- **Learning Rate**: 1e-4
- **Batch Size**: 32 (with gradient accumulation)
- **Precision**: FP16

## Performance

이 모델은 베이스라인 EXAONE 모델 대비 한국어 RAG 태스크에서 향상된 성능을 보입니다.
자세한 평가 결과는 학습 리포지토리를 참고하세요.
"""
        
        # 모델 카드 저장
        with open(f"{model_path}/README.md", "w", encoding="utf-8") as f:
            f.write(model_card_content)
        
        print("📄 모델 카드 생성 완료")
        
        # 파일 업로드
        print("📤 파일 업로드 중... (시간이 걸릴 수 있습니다)")
        
        # 업로드할 파일들
        files_to_upload = [
            "adapter_config.json",
            "adapter_model.safetensors", 
            "tokenizer.json",
            "tokenizer_config.json",
            "model_info.json",
            "README.md"
        ]
        
        for file_name in files_to_upload:
            file_path = os.path.join(model_path, file_name)
            if os.path.exists(file_path):
                api.upload_file(
                    path_or_fileobj=file_path,
                    path_in_repo=file_name,
                    repo_id=repo_name,
                    repo_type="model"
                )
                print(f"   ✅ {file_name} 업로드 완료")
            else:
                print(f"   ⚠️ {file_name} 파일을 찾을 수 없음")
        
        print(f"\n🎉 허깅페이스 업로드 완료!")
        print(f"🔗 모델 URL: https://huggingface.co/{repo_name}")
        print(f"📊 리포지토리 설정:")
        print(f"   - 이름: {repo_name}")
        print(f"   - 공개 여부: {'Private' if private else 'Public'}")
        print(f"   - 모델 타입: LoRA Adapter")
        
        return repo_name
        
    except Exception as e:
        print(f"❌ 업로드 실패: {e}")
        print("💡 문제 해결 방법:")
        print("1. 토큰 권한 확인 (write 권한 필요)")
        print("2. 네트워크 연결 확인")
        print("3. 리포지토리 이름 중복 확인")
        return None

def test_fine_tuned_model(model, tokenizer, test_prompts: List[str]):
    """
    파인튜닝된 모델 테스트
    """
    print("🧪 파인튜닝된 모델 테스트 중...")
    
    model.eval()
    
    for i, prompt in enumerate(test_prompts, 1):
        print(f"\n📝 테스트 {i}:")
        print(f"입력: {prompt[:100]}{'...' if len(prompt) > 100 else ''}")
        
        # 메시지 형태로 변환
        messages = [
            {
                "role": "system",
                "content": "당신은 주어진 컨텍스트를 바탕으로 질문에 정확하고 도움이 되는 답변을 제공하는 AI 어시스턴트입니다."
            },
            {
                "role": "user",
                "content": prompt
            }
        ]
        
        # 입력 토큰화
        input_text = tokenizer.apply_chat_template(
            messages, tokenize=False, add_generation_prompt=True
        )
        
        inputs = tokenizer.encode(input_text, return_tensors="pt")
        if torch.cuda.is_available():
            inputs = inputs.cuda()
        
        # 생성
        with torch.no_grad():
            outputs = model.generate(
                inputs,
                max_new_tokens=150,
                temperature=0.7,
                do_sample=True,
                pad_token_id=tokenizer.eos_token_id,
                eos_token_id=tokenizer.eos_token_id
            )
        
        # 응답 디코딩 (입력 부분 제거)
        generated_text = tokenizer.decode(outputs[0][inputs.shape[1]:], skip_special_tokens=True)
        
        print(f"출력: {generated_text.strip()}")
        print("-" * 80)

# 모델 저장 및 테스트 실행 - 개선된 조건 확인
def check_training_completion():
    """
    학습 완료 상태를 확인하는 함수
    """
    # 변수 존재 확인
    trainer_exists = 'trainer' in locals() or 'trainer' in globals()
    training_result_exists = 'training_result' in locals() or 'training_result' in globals()
    
    print(f"🔍 학습 완료 상태 확인:")
    print(f"  Trainer 존재: {trainer_exists}")
    print(f"  Training Result 존재: {training_result_exists}")
    
    if trainer_exists:
        # trainer 객체가 존재하고 학습이 실행되었는지 확인
        try:
            # 글로벌 스코프에서 trainer 확인
            current_trainer = globals().get('trainer') or locals().get('trainer')
            if current_trainer and hasattr(current_trainer, 'state'):
                print(f"  학습된 스텝 수: {current_trainer.state.global_step}")
                return current_trainer.state.global_step > 0, current_trainer
        except:
            pass
    
    return False, None

# 학습 완료 확인
training_completed, current_trainer = check_training_completion()

if training_completed and current_trainer:
    print("\n✅ 학습이 완료되었습니다. 모델을 저장합니다.")
    
    # 모델 저장
    saved_model_path = save_fine_tuned_model(current_trainer)
    
    # 저장된 파일들의 존재 확인
    import os
    essential_files = [
        f"{saved_model_path}/adapter_config.json",
        f"{saved_model_path}/adapter_model.safetensors"
    ]
    
    print(f"\n🔍 저장된 파일 확인:")
    all_files_exist = True
    for file_path in essential_files:
        if os.path.exists(file_path):
            file_size = os.path.getsize(file_path)
            print(f"  ✅ {os.path.basename(file_path)}: {file_size:,} bytes")
        else:
            print(f"  ❌ {os.path.basename(file_path)}: 파일 없음")
            all_files_exist = False
    
    if all_files_exist:
        print(f"\n🎉 모델 저장 성공! 04번 노트북에서 사용 가능합니다.")
        
        # 자동으로 Hugging Face에 업로드
        print(f"\n🚀 Hugging Face에 업로드 중...")
        try:
            # 허깅페이스에 업로드 (로그인이 되어있다면)
            repo_name = upload_to_huggingface(
                model_path=saved_model_path,
                repo_name="ryanu/my-exaone-raft-model",
                private=False
            )
            
            if repo_name:
                print(f"\n🎉 업로드 완료!")
                print(f"🔗 모델 주소: https://huggingface.co/{repo_name}")
                print(f"💡 Day 2-3 실습에서 이 주소를 사용하여 모델을 불러올 수 있습니다!")
                
                # Day 2-3에서 사용할 모델 주소를 파일로 저장
                model_config = {
                    "model_name": repo_name,
                    "base_model": MODEL_NAME,
                    "upload_timestamp": datetime.now().isoformat(),
                    "usage_instructions": {
                        "load_command": f'PeftModel.from_pretrained(base_model, "{repo_name}")',
                        "description": "Day 1에서 파인튜닝한 EXAONE RAG 모델"
                    }
                }
                
                with open("../day2-RAG/finetuned_model_info.json", "w", encoding="utf-8") as f:
                    json.dump(model_config, f, ensure_ascii=False, indent=2)
                
                print(f"📝 모델 정보 저장: ../day2-RAG/finetuned_model_info.json")
                
        except Exception as e:
            print(f"⚠️ 자동 업로드 실패: {e}")
            print(f"💡 수동으로 업로드하려면 25번 셀을 사용하세요.")
        
        # 테스트 프롬프트 준비
        test_prompts = [
            "다음 컨텍스트들을 참고하여 질문에 답변해주세요.\n\n=== 컨텍스트 ===\n컨텍스트 1: 한국의 수도는 서울입니다.\n컨텍스트 2: 서울은 한강을 중심으로 발달했습니다.\n\n=== 질문 ===\n한국의 수도는 어디인가요?",
            "다음 컨텍스트들을 참고하여 질문에 답변해주세요.\n\n=== 컨텍스트 ===\n컨텍스트 1: 김치는 한국의 전통 발효식품입니다.\n컨텍스트 2: 파스타는 이탈리아의 대표 음식입니다.\n\n=== 질문 ===\n김치는 어떤 음식인가요?",
            "다음 컨텍스트들을 참고하여 질문에 답변해주세요.\n\n=== 컨텍스트 ===\n컨텍스트 1: 태양은 별입니다.\n컨텍스트 2: 지구는 행성입니다.\n\n=== 질문 ===\n바다의 색깔은 무엇인가요?"
        ]
        
        # 모델 테스트
        test_fine_tuned_model(current_trainer.model, tokenizer, test_prompts)
        
        print(f"\n🎉 Day 1 실습 3 완료!")
        print(f"✅ 저장된 모델: {saved_model_path}")
        print(f"✅ Hugging Face 주소: https://huggingface.co/ryanu/my-exaone-raft-model")
        print(f"🔄 다음 단계: 04_evaluation_and_comparison.ipynb에서 성능 평가를 진행하세요!")
    else:
        print(f"\n⚠️ 일부 파일이 저장되지 않았습니다. 다시 저장을 시도하세요.")
        
else:
    print("\n⚠️ 학습이 완료되지 않았습니다.")
    print("💡 학습을 먼저 완료한 후 이 셀을 다시 실행하세요.")
    print("\n📝 학습 상태 확인:")
    if 'trainer' not in locals() and 'trainer' not in globals():
        print("  - Trainer 객체가 생성되지 않았습니다.")
        print("  - 19번 셀(학습 실행)을 먼저 실행하세요.")
    else:
        print("  - Trainer는 존재하지만 학습이 실행되지 않았을 수 있습니다.")

## 13. 허깅페이스 업로드 가이드

### 🌐 파인튜닝된 모델 업로드하기

파인튜닝 완료 후 모델을 허깅페이스에 업로드하여 저장하고 공유할 수 있습니다.

### 📝 준비사항
1. 허깅페이스 계정: https://huggingface.co/join
2. 액세스 토큰 (Write 권한): https://huggingface.co/settings/tokens

### 🔐 로그인 방법
```python
from huggingface_hub import login
login()  # 토큰 입력 프롬프트
```

### 🚀 업로드 실행
아래 셀에서 실제 업로드를 수행할 수 있습니다.

In [ ]:
# 허깅페이스 업로드 실행
# 먼저 로그인이 필요합니다

# 1. 로그인 (아래 주석을 해제하고 실행)
# from huggingface_hub import login
# login()

# 2. 환경변수로 토큰 설정 (선택사항)
# import os
# os.environ['HF_TOKEN'] = 'your_token_here'

# 3. 모델 업로드 실행 (로그인 후 주석 해제)
if 'saved_model_path' in locals():
    print(f"📦 업로드 준비된 모델: {saved_model_path}")
    
    # 업로드 실행 (주석 해제 후 사용)
    """
    repo_name = upload_to_huggingface(
        model_path=saved_model_path,
        repo_name="my-exaone-raft-model",  # 원하는 이름으로 변경
        private=True  # True: 비공개, False: 공개
    )
    print(f"✅ 업로드 완료: https://huggingface.co/{repo_name}")
    """
    
    print("\n💡 업로드하려면:")
    print("1. 위의 login() 주석 해제하고 실행")
    print("2. 토큰 입력")
    print("3. upload_to_huggingface() 주석 해제하고 실행")
    
else:
    print("❌ 저장된 모델이 없습니다. 먼저 파인튜닝을 완료하세요.")

In [ ]:
# 최종 요약 출력
print("🎯 Day 1 실습 3: QLoRA 파인튜닝 완료!")
print("=" * 60)

if 'training_result' in locals():
    # 성공적으로 완료된 경우
    summary = {
        "✅ 모델": MODEL_NAME,
        "📊 학습 샘플": len(train_dataset) if train_dataset else 0,
        "🎯 학습 파라미터": f"{trainable_params:,} ({100*trainable_params/total_params:.3f}%)",
        "⏱️ 학습 스텝": training_result.global_step if 'training_result' in locals() else "N/A",
        "📉 최종 Loss": f"{training_result.training_loss:.4f}" if 'training_result' in locals() else "N/A",
        "💾 저장 위치": saved_model_path if 'saved_model_path' in locals() else "N/A"
    }
    
    print("📋 학습 결과 요약:")
    for key, value in summary.items():
        print(f"  {key}: {value}")
    
    print(f"\n🎉 파인튜닝이 성공적으로 완료되었습니다!")
    print(f"\n📁 생성된 파일들:")
    if 'saved_model_path' in locals():
        print(f"  - {saved_model_path}/adapter_model.safetensors (LoRA 어댑터)")
        print(f"  - {saved_model_path}/adapter_config.json (LoRA 설정)")
        print(f"  - {saved_model_path}/tokenizer.json (토크나이저)")
        print(f"  - {saved_model_path}/model_info.json (모델 정보)")
    
    if 'trainer' in locals():
        print(f"  - {trainer.args.output_dir}/training_analysis.png (학습 분석 그래프)")
        print(f"  - {trainer.args.output_dir}/training_summary.json (학습 요약)")
        print(f"  - {trainer.args.output_dir}/analysis_report.json (분석 리포트)")
    
    print(f"\n🔄 다음 단계:")
    print(f"  1. 04_evaluation_and_comparison.ipynb에서 성능 평가")
    print(f"  2. 파인튜닝 전후 모델 비교")
    print(f"  3. ROUGE, BLEU, 코사인 유사도 측정")
    print(f"  4. 실제 RAG 시나리오에서 테스트")
    
    # 모델 사용 가이드
    print(f"\n📖 모델 로드 방법:")
    print(f"```python")
    print(f"from peft import PeftModel")
    print(f"from transformers import AutoModelForCausalLM, AutoTokenizer")
    print(f"")
    print(f"# 베이스 모델 로드")
    print(f"base_model = AutoModelForCausalLM.from_pretrained('{MODEL_NAME}')")
    print(f"tokenizer = AutoTokenizer.from_pretrained('{MODEL_NAME}')")
    print(f"")
    print(f"# LoRA 어댑터 적용")
    print(f"model = PeftModel.from_pretrained(base_model, '{saved_model_path if 'saved_model_path' in locals() else './your_model_path'}')")
    print(f"```")

else:
    # 학습이 완료되지 않은 경우
    print("⚠️ 파인튜닝이 완전히 완료되지 않았습니다.")
    print(f"\n💡 확인사항:")
    print(f"  1. 데이터가 올바르게 로드되었는지 확인")
    print(f"  2. GPU 메모리가 충분한지 확인")
    print(f"  3. 학습 설정이 적절한지 확인")
    print(f"\n🔄 다시 시도하려면:")
    print(f"  1. 런타임 재시작")
    print(f"  2. 배치 크기 줄이기 (per_device_train_batch_size=1)")
    print(f"  3. 샘플 수 줄이기 (MAX_TRAIN_SAMPLES=200)")

print(f"\n🚀 QLoRA를 활용한 효율적 파인튜닝이 완료되었습니다!")
print(f"🎓 한국어 RAG에 특화된 EXAONE 모델을 얻었습니다!")